# Import Libraries

In [7]:
import pandas as pd
from itertools import zip_longest
from bs4 import BeautifulSoup
from selenium import webdriver

from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [8]:
# code to force show all column
pd.set_option('display.max_columns', None)

# Load Dataset

In [9]:
df = pd.read_csv('rumah123_hyperlink.csv')

# Scrap Web Per Pages

In [10]:
#init webdriver & test BeautifulSoup
driver = webdriver.Chrome()

url="https://www.rumah123.com/jual/rumah/?q=Balikpapan"
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html, "html.parser") #Menentukan parser HTML yang ingin kita gunakan.
print(soup.prettify()[:700]) # memberikan representasi visual dari pohon parse yang dibuat dari konten HTML mentah.

<html lang="id-ID">
 <head>
  <script async="" id="analytics-js" src="https://cdn.segment.com/analytics.js/v1/fpueXeNRJ8uBgCLuBDRDkZFAMBT0yLLb/analytics.min.js" type="text/javascript">
  </script>
  <script>
   partytown = {
                "debug": false,
                "lib": "/asset-core/script/partytown/",
                "logCalls": true,
                "logGetters": true,
                "logImageRequests": true,
                "logMainAccess": true,
                "logScriptExecution": true,
                "logSendBeaconRequests": true,
                "logStackTraces": true,
                "forward": ['dataLayer.push', 'fbq', '_hsq.push', 'ttq.track', 'ttq.page', 'ttq.load'],
 


In [11]:
# prepare variable to save the datas
id = []
judul = []
harga = []
lokasi = []
kamar_tidur = []
kamar_mandi = []
luas_tanah = []
luas_bangunan = []
carport = []
tipe_properti = []
sertifikat = []
daya_listrik = []

kamar_pembantu = []
dapur = []
ruang_makan = []
ruang_tamu = []
kondisi_perabotan = []

# Exterior
jumlah_lantai = []
pemandangan = []

# tentang properti
terjangkau_internet = []
lebar_jalan = []
sumber_air = []
hook = []
kondisi_properti = []
tipe_iklan = []
id_iklan = []

### Changed Logic
1. First initial labels of all possible data
2. Create a dictionary based on the labels
3. Create a Loop to scrap the data and store it in the dictionary
    - First, fill the dictionary with NaN for all key so that when we didn't found the data, it still in the same shape
    - Loop 1: To get the labels1 data (Kamar Tidur, Kamar Mandi, etc)
    - Loop 2: To get the labels2 data (Luas Bangunan, Luas Tanah, etc)

In [12]:
# clean dictionary
data1 = {}
data2 = {}

# define the labels
labels1 = ['Kamar Tidur', 'Kamar Mandi', 'Luas Tanah', 'Luas Bangunan', 'Carport', 'Tipe Properti', 'Sertifikat', 'Daya Listrik']
labels2 = ['Kamar Pembantu', 'Dapur', 'Ruang Makan', 'Ruang Tamu', 'Kondisi Perabotan', 'Jumlah Lantai', 'Pemandangan', 'Terjangkau Internet', 'Lebar Jalan', 'Sumber Air', 'Hook', 'Kondisi Properti', 'Tipe Iklan']

# create dictionaries to store the labels and their values
data1 = {label: [] for label in labels1}
data2 = {label: [] for label in labels2}

loop = 0
print("Progresss: ")

# traverse the hyperlink to get the data needed (test 10 data first)
for i in df['hyperlink']:
    url=i
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # generate random number between 1 and 3
    rand = np.random.randint(1, 12)

    # short pause if loop is a multiple of 5, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 5) == 0) and (loop != 0):
        pause = 15 - rand
        time.sleep(pause)
        # print(f'Long Pause for {pause} seconds...')
    
    # Long pause if loop is a multiple of 19, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 19) == 0) and (loop != 0):
        pause = 59 - rand
        time.sleep(pause)
        print(f'Long Pause for {pause} seconds...')
    
    # initialize all lists in the dictionaries with np.nan for this page
    for label in labels1 + labels2:
        data1[label] = data1.get(label, []) + [np.nan]
        data2[label] = data2.get(label, []) + [np.nan]
    
    get_price_iklan = soup.find_all('div', class_="r123-listing-summary__price")
    # get the span inside the div
    for i in get_price_iklan:
        get_price_iklan = i.find('span')  # use find instead of find_all
        harga.append(get_price_iklan.text)
        
    judul += [ judul.get_text() for judul in soup.find_all( 'h1', {"class":"r123-listing-summary__header-container-title"} ) ]
    lokasi += [ lokasi.get_text() for lokasi in soup.find_all( 'div', {"class":"r123-listing-summary__header-container-address"} ) ]
    
    # ----------------- #

    # first loop
    spec_items = soup.find_all('div', class_="listing-specification-v2__item")
    for item in spec_items:
        label = item.find('span', class_="listing-specification-v2__item-label").text
        value = item.find('span', class_="listing-specification-v2__item-value").text

        # if the label is in the first dictionary, replace the last value (which is np.nan)
        if label in data1:
            data1[label][-1] = value

    # second loop
    interior_items = soup.find_all('div', class_="ui-molecule-accordion-r123")
    for item in interior_items:
        spec_items = item.find_all('div', class_="listing-specification-v2__item")
        for spec_item in spec_items:
            label = spec_item.find('span', class_="listing-specification-v2__item-label").text
            value = spec_item.find('span', class_="listing-specification-v2__item-value").text

            # if the label is in the second dictionary, replace the last value (which is np.nan)
            if label in data2:
                data2[label][-1] = value
   
    #----------------- #
    loop += 1
    
    # print loop progress total number of link divided by 100 (to get the percentage)
    print(f'Done: {loop} / {df["hyperlink"].size}')    

Progresss: 


Done: 1 / 1467
Done: 2 / 1467
Done: 3 / 1467
Done: 4 / 1467
Done: 5 / 1467
Done: 6 / 1467
Done: 7 / 1467
Done: 8 / 1467
Done: 9 / 1467
Done: 10 / 1467
Done: 11 / 1467
Done: 12 / 1467
Done: 13 / 1467
Done: 14 / 1467
Done: 15 / 1467
Done: 16 / 1467
Done: 17 / 1467
Done: 18 / 1467
Done: 19 / 1467
Long Pause for 57 seconds...
Done: 20 / 1467
Done: 21 / 1467
Done: 22 / 1467
Done: 23 / 1467
Done: 24 / 1467
Done: 25 / 1467
Done: 26 / 1467
Done: 27 / 1467
Done: 28 / 1467
Done: 29 / 1467
Done: 30 / 1467
Done: 31 / 1467
Done: 32 / 1467
Done: 33 / 1467
Done: 34 / 1467
Done: 35 / 1467
Done: 36 / 1467
Done: 37 / 1467
Done: 38 / 1467
Long Pause for 53 seconds...
Done: 39 / 1467
Done: 40 / 1467
Done: 41 / 1467
Done: 42 / 1467
Done: 43 / 1467
Done: 44 / 1467
Done: 45 / 1467
Done: 46 / 1467
Done: 47 / 1467
Done: 48 / 1467
Done: 49 / 1467
Done: 50 / 1467
Done: 51 / 1467
Done: 52 / 1467
Done: 53 / 1467
Done: 54 / 1467
Done: 55 / 1467
Done: 56 / 1467
Done: 57 / 1467
Long Pause for 57 seconds...
Done: 58 /

In [13]:
# get the intersection of the keys in data1 and data2
duplicate_labels = set(data1.keys()) & set(data2.keys())

# for each duplicate label, remove it from data2
for label in duplicate_labels:
    del data2[label]

# convert the dictionaries to dataframes
judull = pd.DataFrame(judul, columns=['Judul'])
hargaa = pd.DataFrame(harga, columns=['Harga'])
lokasia = pd.DataFrame(lokasi, columns=['Lokasi'])

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# concatenate the dataframes
df_fix = pd.concat([judull, hargaa, lokasia, df1, df2], axis=1)
print(df_fix.shape)
df_fix

(1467, 24)


,Judul,Harga,Lokasi,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
0,RUMAH 2 LANTAI TENGAH KOTA PERUMAHAN BUKIT DAM...,"Rp 1,5 Miliar","Balikpapan Selatan, Balikpapan",5,3,144 m²,207 m²,1,Rumah,SHM - Sertifikat Hak Milik,4400 Watt,1,1,Tidak,Tidak,Semi Furnished,2,Pemukiman Warga,Ya,2 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
1,Rumah Luas Lokasi Strategis di Gunung Samarind...,"Rp 3,1 Miliar","Balikpapan Utara, Balikpapan",3,2,534 m²,200 m²,3,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,1,Ya,Ya,NaN,1,NaN,Ya,NaN,PAM atau PDAM,Tidak,Bagus,Dijual
2,Rumah Minimalis Baru Renovasi Untuk Keluarga M...,Rp 600 Juta,"Balikpapan Utara, Balikpapan",2,2,80 m²,140 m²,1,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,1,Ya,Ya,Semi Furnished,1,NaN,Ya,NaN,PAM atau PDAM,Tidak,Bagus,Dijual
3,"DIJUAL CEPAT RUMAH DI JALAN SWADAYA, KARANG RE...",Rp 510 Juta,"Balikpapan Tengah, Balikpapan",3,2,201 m²,150 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,Tidak,Tidak,Unfurnished,1,NaN,Tidak,NaN,NaN,Tidak,Bagus,Dijual
4,Rumah Minimalis Modern Smart Home Tanah Luas,Rp 850 Juta,"Balikpapan Utara, Balikpapan",3,3,120 m²,65 m²,1,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,1,Ya,Ya,Unfurnished,2,Pemukiman Warga,Ya,NaN,Sumur Bor,Tidak,Baru,Dijual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,NaN,NaN,NaN,2,1,120 m²,36 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,900 Watt,NaN,NaN,Tidak,Tidak,Unfurnished,1,Pemukiman Warga,Ya,NaN,NaN,Tidak,Butuh Renovasi,Dijual
1463,NaN,NaN,NaN,3,3,374 m²,200 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,6600 Watt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1464,NaN,NaN,NaN,2,1,72 m²,45 m²,1,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,1,Tidak,Ya,Unfurnished,1,Pemukiman Warga,Ya,NaN,PAM atau PDAM,Ya,Baru,Dijual
1465,NaN,NaN,NaN,5,2,403 m²,200 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual


In [16]:
# to csv
df_fix.to_csv('rumah123_data_single_raw.csv', index=False)

In [14]:
# concat the dataframe above with the original dataframe (10 data)
df_final_10 = pd.concat([df, df_fix], axis=1)
df_final_10

,judul,hyperlink,img_hyperlinks,Judul,Harga,Lokasi,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
0,RUMAH 2 LANTAI TENGAH KOTA PERUMAHAN BUKIT DAM...,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,RUMAH 2 LANTAI TENGAH KOTA PERUMAHAN BUKIT DAM...,"Rp 1,5 Miliar","Balikpapan Selatan, Balikpapan",5,3,144 m²,207 m²,1,Rumah,SHM - Sertifikat Hak Milik,4400 Watt,1,1,Tidak,Tidak,Semi Furnished,2,Pemukiman Warga,Ya,2 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
1,Rumah Luas Lokasi Strategis di Gunung Samarind...,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123-images/720x4...,Rumah Luas Lokasi Strategis di Gunung Samarind...,"Rp 3,1 Miliar","Balikpapan Utara, Balikpapan",3,2,534 m²,200 m²,3,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,1,Ya,Ya,NaN,1,NaN,Ya,NaN,PAM atau PDAM,Tidak,Bagus,Dijual
2,Rumah Minimalis Baru Renovasi Untuk Keluarga M...,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123-images/720x4...,Rumah Minimalis Baru Renovasi Untuk Keluarga M...,Rp 600 Juta,"Balikpapan Utara, Balikpapan",2,2,80 m²,140 m²,1,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,1,Ya,Ya,Semi Furnished,1,NaN,Ya,NaN,PAM atau PDAM,Tidak,Bagus,Dijual
3,"DIJUAL CEPAT RUMAH DI JALAN SWADAYA, KARANG RE...",https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,"DIJUAL CEPAT RUMAH DI JALAN SWADAYA, KARANG RE...",Rp 510 Juta,"Balikpapan Tengah, Balikpapan",3,2,201 m²,150 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,Tidak,Tidak,Unfurnished,1,NaN,Tidak,NaN,NaN,Tidak,Bagus,Dijual
4,Rumah Minimalis Modern Smart Home Tanah Luas,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123-images/720x4...,Rumah Minimalis Modern Smart Home Tanah Luas,Rp 850 Juta,"Balikpapan Utara, Balikpapan",3,3,120 m²,65 m²,1,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,1,Ya,Ya,Unfurnished,2,Pemukiman Warga,Ya,NaN,Sumur Bor,Tidak,Baru,Dijual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,Rumah Murah Harga Nego Dalam Perumahan,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,2,1,120 m²,36 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,900 Watt,NaN,NaN,Tidak,Tidak,Unfurnished,1,Pemukiman Warga,Ya,NaN,NaN,Tidak,Butuh Renovasi,Dijual
1463,Perumahan Elit Pupuk Kaltim,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,3,3,374 m²,200 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,6600 Watt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1464,Rumah Baru Minimalis Kpr Tanpa Dp,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,2,1,72 m²,45 m²,1,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,1,Tidak,Ya,Unfurnished,1,Pemukiman Warga,Ya,NaN,PAM atau PDAM,Ya,Baru,Dijual
1465,BALIKPAPAN POKA (Y140),https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,5,2,403 m²,200 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual


In [17]:
# check null values
df_final_10.isnull().sum()

judul                     0
hyperlink                 0
img_hyperlinks            0
Judul                    14
Harga                    14
Lokasi                   14
Kamar Tidur              64
Kamar Mandi              65
Luas Tanah               14
Luas Bangunan            14
Carport                1205
Tipe Properti            14
Sertifikat               15
Daya Listrik             65
Kamar Pembantu         1224
Dapur                  1235
Ruang Makan             748
Ruang Tamu              744
Kondisi Perabotan      1072
Jumlah Lantai          1060
Pemandangan            1123
Terjangkau Internet     745
Lebar Jalan            1133
Sumber Air             1124
Hook                    745
Kondisi Properti        960
Tipe Iklan               14
dtype: int64

In [19]:
# pinrt row with null judul value   
df_final_10[df_final_10['Judul'].isnull()]

,judul,hyperlink,img_hyperlinks,Judul,Harga,Lokasi,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
1453,CANTIK TIGA KAMAR SIAP HUNI,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,3,1,105 m²,105 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1454,CANTIK BARU JADI DEKET RS SILOAM,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,3,3,326 m²,300 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,4400 Watt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1455,BESAR DAN MEGAH DALAM CLUSTER,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,5,4,200 m²,298 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,5500 Watt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1456,SEGERA MILIKI HARGA TERBAIK,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,3,2,162 m²,105 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1457,DI JUAL RUMAH DUA LANTAI DI KAWASAN PERUMAHAN ...,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,3,2,112 m²,85 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual
1458,Rumah cantik Perumnas,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,3,2,149 m²,205 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,NaN,NaN,NaN,NaN,NaN,Unfurnished,1,NaN,NaN,NaN,NaN,NaN,Bagus,Dijual
1459,KAWASAN HUNIAN NYAMAN DAN AMAN DI TIMUR KOTA ...,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,2,1,104 m²,45 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1460,Bagus Terawat dan Besar,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,3,2,120 m²,128 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1461,RUMAH LUAS TERAWAT DALAM PERUMAHAN DI BALIKPAPAN,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,4,2,215 m²,300 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1462,Rumah Murah Harga Nego Dalam Perumahan,https://www.rumah123.com//properti/balikpapan/...,https://picture.rumah123.com/r123/720x420-crop...,NaN,NaN,NaN,2,1,120 m²,36 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,900 Watt,NaN,NaN,Tidak,Tidak,Unfurnished,1,Pemukiman Warga,Ya,NaN,NaN,Tidak,Butuh Renovasi,Dijual


In [15]:
# to csv
df_final_10.to_csv('rumah123_fix_data.csv', index=False)